In [1]:
%%time
from functools import partial
from collections import defaultdict
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product
import time

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 5.76 s, sys: 860 ms, total: 6.62 s
Wall time: 6.09 s


In [2]:
detector = FastAIScreenDetector()
method_params = list(detector.methods.keys())
train_params = list(detector.train_params.keys())
all_params = train_params + method_params

base = [True]
base_lower_bounds = [7]
base_upper_bounds = [70]
base_factors = [[0.1, 0.01]]

erode_dilate = [True]
erode_dilate_lower_bounds = [7]
erode_dilate_upper_bounds = [70]
erode_dilate_factors = [[0.1, 0.01]]
erode_dilate_iterations = [40, 100]

ratio_split = [True]
ratio_split_lower_bounds = [0.7, 0.9]
ratio_split_upper_bounds = [1.5]

methods_values = [base] + [erode_dilate] + [ratio_split] + [base_lower_bounds] + [base_upper_bounds] \
        + [base_factors] + [erode_dilate_lower_bounds] + [erode_dilate_upper_bounds] + [erode_dilate_factors] \
        + [erode_dilate_iterations] + [ratio_split_lower_bounds] + [ratio_split_upper_bounds]

batch_size = [8]
resize_factor = [2]
frozen_epochs = [2, 6, 9]
unfrozen_epochs = [3, 7, 10]
frozen_lr = [1e-3, 1e-4]
unfrozen_lr = [slice(1e-4, 2e-4)]

train_params_values = [batch_size] + [resize_factor] + [frozen_epochs] + [unfrozen_epochs] + [frozen_lr] + [unfrozen_lr]

In [3]:
method_settings = list(product(*methods_values))
train_settings = list(product(*train_params_values))
all_lectures = [video.filename for video in ALL_VIDEOS]
all_frames = [frame for video in ALL_VIDEOS for frame in video]
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}
test_lectures = ['PB069-D2-20140305.mp4']
test_frames = [frame for lecture in test_lectures for frame in all_frames_grouped_by_videos[lecture]]
actual_detector = AnnotatedSampledVideoScreenDetector()

In [4]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=123)
df_best_models = pd.DataFrame(columns=all_params + ['iou', 'wrong_count'])
for i, split in tqdm(enumerate(kf.split(all_lectures))):
    other_lectures = [all_lectures[index] for index in split[0]]
    test_lectures = [all_lectures[index] for index in split[1]]
    
    # Model selection
    df_all = pd.DataFrame(columns=all_params + ['iou', 'wrong_count', 'kfold_split'])
    
    for train_setting in tqdm(train_settings):
        train_params_dict = dict(zip(train_params, train_setting))
        for j, split in enumerate(kf.split(other_lectures)):
            train_lectures = [other_lectures[index] for index in split[0]]
            valid_lectures = [other_lectures[index] for index in split[1]]
            valid_frames = [frame for lecture in valid_lectures for frame in all_frames_grouped_by_videos[lecture]]

            filtered_by = lambda name: any([lecture in str(name) for lecture in train_lectures + valid_lectures])  \
                            and 'frame' in str(name)
            split_by = lambda name: any([lecture in str(name) for lecture in valid_lectures])
            
            detector = FastAIScreenDetector(train_params=train_params_dict, methods=None, filtered_by=filtered_by,
                                        valid_func=split_by, progressbar=False, device='cuda')
        
            detector.train()
            
            actuals = [actual_detector.detect(frame) for frame in valid_frames]
            sem_preds = detector.semantic_segmentation_batch(valid_frames)
            
            for i, method_setting in enumerate(method_settings):    
                preds = detector.post_processing_batch(sem_preds, valid_frames, dict(zip(method_params, method_setting)))
                wrong_count, ious, _ = evaluate(actuals, preds)
                
                iou_score = np.nanmean(ious)
                wrong_count = len(wrong_count)
                df_all.loc[len(df_all)] = train_setting + method_setting + (iou_score, wrong_count, j)
    
    unhashable_columns = ['frozen_lr', 'unfrozen_lr', 'base_factors', 'erode_dilate_factors']
    df_all[unhashable_columns] = df_all[unhashable_columns].astype(str)
    df_all['wrong_count'] = df_all['wrong_count'].astype(int)
    
    best_params = df_all.groupby(train_params + method_params).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name
    converted_params = []
    for i, par in enumerate(best_params):
        if isinstance(par, np.int64) or isinstance(par, np.float64):
            converted_params.append(par.item())
        else:
            converted_params.append(par)
    best_params = tuple(converted_params)
    
    best_methods = dict(zip(method_params, best_params[-len(method_params):]))
    best_train_params_dict = dict(zip(train_params, best_params[:len(train_params)]))
    best_train_params_dict['frozen_lr'] = eval(best_train_params_dict['frozen_lr'])
    best_train_params_dict['unfrozen_lr'] = eval(best_train_params_dict['unfrozen_lr'])
    best_methods['base_factors'] = eval(best_methods['base_factors'])
    best_methods['erode_dilate_factors'] = eval(best_methods['erode_dilate_factors'])
    
    
    filtered_by = lambda name: 'frame' in str(name)
    split_by = lambda name: any([lecture in str(name) for lecture in test_lectures])
    
    best_detector = FastAIScreenDetector(train_params=best_train_params_dict, methods=best_methods, 
                                         filtered_by=filtered_by, valid_func=split_by, progressbar=True, device='cuda')
    best_detector.train()

    actuals = [actual_detector.detect(frame) for frame in valid_frames]
    preds = [best_detector.detect(frame) for frame in valid_frames]
    wrong_count, iou, _ = evaluate(actuals, preds)
    iou_score = np.nanmean(ious)
    wrong_count = len(wrong_count)
    df_best_models.loc[len(df_best_models)] = train_setting + method_setting + (iou_score, wrong_count)
    df_best_models.to_csv('cross_validation_results.csv', index=False)
df_best_models.to_csv('cross_validation_results.csv')

0it [00:00, ?it/s]
  0%|          | 0/18 [00:00<?, ?it/s]


  6%|▌         | 1/18 [06:37<1:52:41, 397.76s/it]


 11%|█         | 2/18 [12:53<1:44:20, 391.30s/it]


 28%|██▊       | 5/18 [48:43<2:14:38, 621.40s/it]


 33%|███▎      | 6/18 [1:02:58<2:18:18, 691.56s/it]

...


100%|██████████| 18/18 [4:43:16<00:00, 944.24s/it] 


epoch,train_loss,valid_loss,acc,dice,iou,time
0,0.232944,0.109097,0.946056,0.892640,0.819051,00:18
1,0.131544,0.060144,0.974024,0.942271,0.895718,00:17


epoch,train_loss,valid_loss,acc,dice,iou,time
0,0.082832,0.156206,0.951223,0.903428,0.839400,00:18
1,0.068399,0.124386,0.954879,0.908860,0.845913,00:18
2,0.048719,0.053915,0.979603,0.953867,0.914846,00:18


5it [23:26:47, 16881.43s/it]

CPU times: user 12h 19min 6s, sys: 3h 15min 10s, total: 15h 34min 17s
Wall time: 23h 26min 47s
